## Introduction

In this project I will use Ye

## Import Necessary libraries

In [48]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
import pandas as pd
import numpy as np

### Get  data

In [49]:
df = pd.read_csv('train.csv', sep = '|', names = ['stars', 'text'], error_bad_lines=False)

In [50]:
df= df.dropna()
df = df[df.stars.apply(lambda x: x.isnumeric())]
df = df[df.stars.apply(lambda x: x !="")]

In [51]:
df = df[df.text.apply(lambda x: x !="")]

In [52]:
df.describe()

,stars,text
count,1673870,1673870
unique,5,1673452
top,5,Good stuff
freq,709732,6


In [53]:
df.head()

,stars,text
0,5,The minute I realized that Conflict was a bloc...
2,5,I love Conflict Kitchen. The food is fantasti...
3,4,Holy moly! I'm addicted!\n\nI first heard of C...
4,4,"Had some great Persian food, though it was mor..."
5,4,Yummy food. Good prices. Encourages me to try ...


In [54]:
labels = df['stars'].map(lambda x : 1 if int(x) > 3 else 0)

In [72]:
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=50)

In [73]:
print(data.shape)

(1673870, 50)


## Build LSTM model

In [13]:
model = Sequential()
model.add(Embedding(20000, 100, input_length=50))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
model.fit(data, np.array(labels), validation_split=0.4, epochs=3)

Train on 1004322 samples, validate on 669548 samples
Epoch 1/3
1004322/1004322 [==============================] - 7913s - loss: 0.2875 - acc: 0.8776 - val_loss: 0.2553 - val_acc: 0.8934
Epoch 2/3
1004322/1004322 [==============================] - 7931s - loss: 0.2454 - acc: 0.8978 - val_loss: 0.2469 - val_acc: 0.8975
Epoch 3/3
1004322/1004322 [==============================] - 11974s - loss: 0.2291 - acc: 0.9057 - val_loss: 0.2530 - val_acc: 0.8977


NameError: name 'padded_docs' is not defined

## Adding 1D CNN layer
Our LSTM model worked well enough, but it takes forever to train 3 epochs. One way to speed up the training time is to improve our network architecture and add a “Convolutional” layer. Convolutional Neural Networks (CNNs) come from image processing. They pass a “filter” over the data, and calculate a higher-level representation. They have been shown to work surprisingly well for text, even though they have none of the sequence processing ability of LSTMs.

In [59]:
def create_conv_model():
    model_conv = Sequential()
    model_conv.add(Embedding(vocabulary_size, 100, input_length=50))
    model_conv.add(Dropout(0.2))
    model_conv.add(Conv1D(64, 5, activation='relu'))
    model_conv.add(MaxPooling1D(pool_size=4))
    model_conv.add(LSTM(100))
    model_conv.add(Dense(1, activation='sigmoid'))
    model_conv.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_conv 

In [60]:
model_conv = create_conv_model()
model_conv.fit(data, np.array(labels), validation_split=0.4, epochs = 3)

Train on 1004322 samples, validate on 669548 samples
Epoch 1/3
1004322/1004322 [==============================] - 3260s - loss: 0.2953 - acc: 0.8735 - val_loss: 0.2727 - val_acc: 0.8840
Epoch 2/3
1004322/1004322 [==============================] - 3256s - loss: 0.2576 - acc: 0.8918 - val_loss: 0.2658 - val_acc: 0.8884
Epoch 3/3
1004322/1004322 [==============================] - 6413s - loss: 0.2406 - acc: 0.9000 - val_loss: 0.2677 - val_acc: 0.8884


In [28]:
df_save = pd.DataFrame(data)
df_label = pd.DataFrame(np.array(labels))

In [29]:
result = pd.concat([df_save, df_label], axis = 1)

In [31]:
result.to_csv('train_dense_word_vectors.csv', index=False)

### Use Pre-Trained GloVe Embedding

In [61]:
embeddings_index = dict()
f = open('glove.6B/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [66]:
print(embeddings_index['the'])

[-0.038194   -0.24487001  0.72812003 -0.39961001  0.083172    0.043953
 -0.39140999  0.3344     -0.57545     0.087459    0.28786999 -0.06731
  0.30906001 -0.26383999 -0.13231    -0.20757     0.33395001 -0.33848
 -0.31742999 -0.48335999  0.1464     -0.37303999  0.34577     0.052041
  0.44946    -0.46970999  0.02628    -0.54154998 -0.15518001 -0.14106999
 -0.039722    0.28277001  0.14393     0.23464    -0.31020999  0.086173
  0.20397     0.52623999  0.17163999 -0.082378   -0.71787    -0.41531
  0.20334999 -0.12763     0.41367     0.55186999  0.57907999 -0.33476999
 -0.36559001 -0.54856998 -0.062892    0.26583999  0.30204999  0.99774998
 -0.80480999 -3.0243001   0.01254    -0.36941999  2.21670008  0.72201002
 -0.24978     0.92136002  0.034514    0.46744999  1.10790002 -0.19358
 -0.074575    0.23353    -0.052062   -0.22044     0.057162   -0.15806
 -0.30798    -0.41624999  0.37972     0.15006    -0.53211999 -0.20550001
 -1.25259995  0.071624    0.70564997  0.49744001 -0.42063001  0.26148
 -

In [105]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocabulary_size, 100))

for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [106]:
model_glove = Sequential()
model_glove.add(Embedding(vocabulary_size, 100, input_length=50, weights=[embedding_matrix], trainable=False))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(64, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(LSTM(100))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model_glove.fit(data, np.array(labels), validation_split=0.4, epochs = 3)

Train on 1004322 samples, validate on 669548 samples
Epoch 1/3
1004320/1004322 [============================>.] - ETA: 0s - loss: 0.3780 - acc: 0.8262

In [ ]:
model = Word2Vec.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True, norm_only=True)
# getting word vectors of a word
dog = model['dog']
#performing king queen magic
print(model.most_similar(positive=['woman', 'king'], negative=['man']))

#picking odd one out
print(model.doesnt_match("breakfast cereal dinner lunch".split()))

#printing similarity index
print(model.similarity('woman', 'man'))

In [ ]:
sentence=[[‘Neeraj’,’Boy’],[‘Sarwan’,’is’],[‘good’,’boy’]]
model = gensim.models.Word2Vec(sentence, min_count=1,size=300,workers=4)
print(model.similarity('woman', 'man'))